# Optimal neighborhood size selection

In [ ]:
from pathlib import Path

import matplotlib.pyplot as plt
import numpy as np
from scipy import spatial
from tqdm.auto import tqdm, trange

In [ ]:
def show_point_cloud(
    points: np.ndarray,
    n: np.ndarray = None,
    elev: float = 0,
    azim: float = 0,
    **kwargs: dict,
) -> tuple[plt.Figure, plt.Axes]:
    """Visualize a 3D point cloud.

    Args:
        points (np.ndarray): Point cloud of shape (N, 3).
        n (np.ndarray, optional): Normals of shape (N, 3). Defaults to None.
        elev (float, optional): Elevation angle in degrees. Defaults to 0.
        azim (float, optional): Azimuth angle in degrees. Defaults to 0.
        **kwargs: Additional keyword arguments for scatter plot.

    Returns:
        fig, ax: Matplotlib figure and axes objects.

    """
    if isinstance(n, np.ndarray):
        n = np.divide(
            n,
            np.tile(
                np.expand_dims(np.sqrt(np.sum(np.square(n), axis=1)), axis=1),
                [1, 3],
            ),
        )
        c = (127.5 + 127.5 * n) / 255
        cbar = True
        alpha = 1.0
    else:
        c = "k"
        cbar = False
        alpha = 0.05
    fig = plt.figure(figsize=(5, 5))
    ax = plt.axes(projection="3d")
    s = ax.scatter(*points.T, s=0.25, c=c, alpha=alpha, **kwargs)
    if cbar:
        cbar = fig.colorbar(s, ax=ax, pad=0, shrink=0.5)
        cbar.solids.set(alpha=1)
    ax.set_box_aspect([1, 1, 1])
    ax.axis("off")
    ax.view_init(elev=elev, azim=azim, vertical_axis="y")
    return fig, ax

In [ ]:
def estimate_normals(points: np.ndarray, k: int | np.ndarray) -> np.ndarray:
    """Return estimated normals for a point cloud.

    Args:
        points (np.ndarray): Point cloud of shape (N, 3).
        k (int | np.ndarray): Number of neighbors to use for normal estimation.

    Returns:
        np.ndarray: Estimated normals of shape (N, 3).

    """
    # Create a kd-tree for quick nearest-neighbor lookup
    tree = spatial.KDTree(points)
    n = np.empty_like(points)
    if isinstance(k, int):
        k = np.full((points.shape[0],), k)
    for i, p in enumerate(tqdm(points)):
        # Extract the local neighborhood
        _, idx = tree.query([p], k=k[i])
        nbhd = points[idx.flatten()]

        # Extract an eigenvector with smallest associated eigenvalue
        nbhd_c = nbhd - np.mean(nbhd, axis=0)
        c = (nbhd_c.T @ nbhd_c) / k[i]
        u, s, _ = np.linalg.svd(c)
        n[i, :] = u[:, np.argmin(s)]
    return n

In [ ]:
def angle_error(
    n_estim: np.ndarray,
    n_gt: np.ndarray,
    orient: bool = True,
) -> np.ndarray:
    """Return angle error between estimated and ground-truth normals.

    Args:
        n_estim (np.ndarray): Estimated normals of shape (N, 3).
        n_gt (np.ndarray): Ground-truth normals of shape (N, 3).
        orient (bool, optional): Whether to consider normal orientation.

    Returns:
        np.ndarray: Angle errors in degrees of shape (N,).

    """
    size = n_gt.shape[0]
    if orient:
        dot = np.sum(n_estim * n_gt, axis=1)
    else:
        dot = np.sum(np.abs(n_estim) * np.abs(n_gt), axis=1)
    dot = np.where(dot > 1, 1, dot)
    dot = np.where(dot < -1, -1, dot)
    if np.sum(dot) / size == 1:  # all normals matched -> no error
        return 0
    return np.arccos(dot) * 180 / np.pi

In [ ]:
def rms_angle_error(
    n_estim: np.ndarray,
    n_gt: np.ndarray,
    orient: bool = False,
) -> float:
    """Return root mean square angle error between estimated and ground-truth normals.

    Args:
        n_estim (np.ndarray): Estimated normals of shape (N, 3).
        n_gt (np.ndarray): Ground-truth normals of shape (N, 3).
        orient (bool, optional): Whether to consider normal orientation.

    Returns:
        float: Root mean square angle error in degrees.

    """
    err = angle_error(n_estim, n_gt, orient)
    return np.sqrt(np.mean(err**2))

In [ ]:
def get_opt_nn(
    points: np.ndarray,
    scale: tuple,
    step: int = 1,
    solver: str = "eigenentropy",
) -> list[int]:
    """Return optimal neighborhood size for each point in the point cloud.

    https://isprs-annals.copernicus.org/articles/II-3/181/2014/

    Args:
        points (np.ndarray): Point cloud of shape (N, 3).
        scale (tuple): Minimum and maximum neighborhood sizes (k_min, k_max).
        step (int, optional): Step size for neighborhood sizes. Defaults to 1.
        solver (str, optional): Method to compute entropy. Options:
          - "entropy"
          - "eigenentropy" (default)

    Returns:
        list: Optimal neighborhood size for each point.

    """
    size = points.shape[0]
    k_min, k_max = scale
    k_scale = np.arange(k_min, k_max + 1, step, dtype="int64")
    tree = spatial.KDTree(points)
    _, idx = tree.query(points, k=k_max, workers=-1)
    opt_nn = []
    for p_idx in trange(size):
        entropy = []
        for k in k_scale:
            # Eigenvalues from the neighborhood covariance
            nbhd = points[idx[p_idx, :k]]
            nbhd_c = nbhd - np.mean(nbhd, axis=0)
            cov = np.cov(nbhd_c, rowvar=False)
            e, _ = np.linalg.eig(cov)
            e = np.maximum(e, 1e-8)  # For stability sake
            e = e[np.argsort(e)[::-1]]  # Sort in descending order
            e /= np.sum(e)  # Normalization
            if solver == "entropy":
                lin = (e[0] - e[1]) / e[0]  # Linearity
                pln = (e[1] - e[2]) / e[0]  # Planarity
                sct = e[2] / e[0]  # Scattering
                entropy.append(
                    -lin * np.log(lin) - pln * np.log(pln) - sct * np.log(sct),
                )
            elif solver == "eigenentropy":
                entropy.append(
                    -e[0] * np.log(e[0]) - e[1] * np.log(e[1]) - e[2] * np.log(e[2]),
                )
            else:
                raise NotImplementedError
        opt_nn.append(k_scale[np.argmin(entropy)])
    return opt_nn

In [ ]:
# Load bunny point cloud
bunny = Path("data") / "bunny" / "bunny100k"
points = np.loadtxt(f"{bunny}.xyz")
n = np.loadtxt(f"{bunny}.normals")
fig, ax = show_point_cloud(points, n)

In [ ]:
# Estimate normals with optimal neighborhood sizes
scale = (10, 100)
step = 1
opt_nn = get_opt_nn(points, scale, step, solver="eigenentropy")
n_opt = estimate_normals(points, opt_nn)
theta = rms_angle_error(n_opt, n)
theta